## CNL (In context learning'de ensemble)

In [ ]:
!pip install torch transformers datasets

  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.4.5-cp311-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ahmed\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
!pip3 install datasets

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following de

### Imports, Set Path and Constant Values

In [1]:
import sys
import os

# Set Constants
# If use Google Colab
use_colab = True
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT_PATH = "/content/drive/MyDrive/Colab Notebooks/CNL"  # Google Drive için kök dizin
    sys.path.append(os.path.join(ROOT_PATH, 'src'))
    device = "cuda"
else:
    ROOT_PATH = "G:\Drive'ım\Colab Notebooks\CNL"
    device = "cpu"

CACHE_PATH = os.path.join(ROOT_PATH, 'cache')
DATA_PATH = os.path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')

# Create Folders
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(CACHE_PATH, exist_ok=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import json
#from src.utils import missing_values
from utils import missing_values
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer
#from datasets import load_dataset

### Load Datasets & Show Metadata, Shape and Missing Table

#### ARC-TR

In [ ]:
arc_dataset = load_dataset("malhajar/arc-tr-v0.2", split="test")

In [ ]:
df_arc = arc_dataset.to_pandas()
df_arc.head()

,id,question,choices,answerKey
0,Mercury_7175875,"Bir gökbilimci, bir göktaşının çarpmasından so...","{'label': ['A', 'B', 'C', 'D'], 'text': ['Geze...",C
1,Mercury_SC_409171,"Bir grup mühendis, farklı bina tasarımlarının ...","{'label': ['A', 'B', 'C', 'D'], 'text': ['bina...",B
2,Mercury_SC_408547,Fotosentez sürecinin nihai sonucu şeker ve oks...,"{'label': ['A', 'B', 'C', 'D'], 'text': ['Kimy...",C
3,Mercury_407327,"Bir fizikçi, bir arabanın bir rampadan atlamak...","{'label': ['A', 'B', 'C', 'D'], 'text': ['kont...",D
4,MCAS_2006_9_44,Bir astronot Ay'a 1.0 kg'lık bir cisim ve 5.0 ...,"{'label': ['A', 'B', 'C', 'D'], 'text': ['Her ...",D


In [ ]:
df_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1172 non-null   object
 1   question   1172 non-null   object
 2   choices    1172 non-null   object
 3   answerKey  1172 non-null   object
dtypes: object(4)
memory usage: 36.8+ KB


In [ ]:
df_arc.shape

(1172, 4)

In [ ]:
missing_values(df=df_arc, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
id,0,0.0
question,0,0.0
choices,0,0.0
answerKey,0,0.0


#### Hellaswag-TR

In [ ]:
hellaswag_dataset = load_dataset("malhajar/hellaswag-tr", split="train")

C:\Users\ahmed\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahmed\.cache\huggingface\hub\datasets--malhajar--hellaswag-tr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 

In [ ]:
df_hellas = hellaswag_dataset.to_pandas()
df_hellas.head()

,ind,activity_label,ctx_a,ctx_b,ctx,endings,source_id,split,split_type,label
0,8,Kurabiye pişirmek,"Beyaz üniformalı bir kadın şef, büyük bir mutf...",tavalar,"Beyaz üniformalı bir kadın şef, büyük bir mutf...","[yumurta sarısı ve kabartma tozu içerir., daha...",activitynet~v_-2dxp-mv2zo,train,indomain,3
1,9,Kurabiye pişirmek,"Beyaz üniformalı bir kadın şef, büyük bir mutf...",Bir bıçak,"Beyaz üniformalı bir kadın şef, büyük bir mutf...",[bir tahta üzerinde hareket ettiği ve içindeki...,activitynet~v_-2dxp-mv2zo,train,indomain,3
2,12,Kurabiye pişirmek,Fırına bir tepsi patates konulur ve çıkarılır....,büyük bir tepsi et,Fırına bir tepsi patates konulur ve çıkarılır....,"[fırında patatesin üzerine yerleştirilir., , l...",activitynet~v_-2dxp-mv2zo,train,indomain,3
3,27,Saç kestirmek,Ortadaki adam mavi gömlek giyen kişinin saç st...,mavi gömlekli adam,Ortadaki adam mavi gömlek giyen kişinin saç st...,[mavi gömlek giyen kişinin saçını kesen sünger...,activitynet~v_-JqLjPz-07E,train,indomain,2
4,29,Saç kestirmek,Mavi gömlekli adam lavabonun yanındaki sandaly...,o,Mavi gömlekli adam lavabonun yanındaki sandaly...,"[su musluğunu lavaboya düşürür., sonra lavaboy...",activitynet~v_-JqLjPz-07E,train,indomain,3


In [ ]:
df_hellas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39902 entries, 0 to 39901
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ind             39902 non-null  int32 
 1   activity_label  39902 non-null  object
 2   ctx_a           39902 non-null  object
 3   ctx_b           39902 non-null  object
 4   ctx             39902 non-null  object
 5   endings         39902 non-null  object
 6   source_id       39902 non-null  object
 7   split           39902 non-null  object
 8   split_type      39902 non-null  object
 9   label           39902 non-null  object
dtypes: int32(1), object(9)
memory usage: 2.9+ MB


In [ ]:
df_hellas.shape

(39902, 10)

In [ ]:
missing_values(df=df_hellas, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
ind,0,0.0
activity_label,0,0.0
ctx_a,0,0.0
ctx_b,0,0.0
ctx,0,0.0
endings,0,0.0
source_id,0,0.0
split,0,0.0
split_type,0,0.0


#### GSM8K-TR

In [ ]:
gsm8k_dataset = load_dataset("malhajar/gsm8k-tr", "main", split="train")

In [ ]:
df_gsm8k = gsm8k_dataset.to_pandas()
df_gsm8k.head()

,question,answer
0,Weng bebek bakıcılığından saatte 12 dolar kaza...,"Weng dakikada 12/60 = $<<12/60=0,2>>0,2 kazanı..."
1,"Betty, maliyeti 100 dolar olan yeni bir cüzdan...",Başlangıçta Betty'nin elinde sadece 100/2 = $<...
2,Julie 120 sayfalık bir kitap okuyor. Dün 12 sa...,Maila bugün 12 x 2 = <<12*2=24>>24 sayfa okudu...
3,James haftada iki kez 2 farklı arkadaşına 3 sa...,Her arkadaşına haftada 3*2=<<3*2=6>>6 sayfa ya...
4,Mark'ın çiçeklerle dolu bir bahçesi var. İçine...,Sarı çiçeklerden 80/100 * 10 = <<80/100*10=8>>...


In [ ]:
df_gsm8k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7472 entries, 0 to 7471
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  7472 non-null   object
 1   answer    7472 non-null   object
dtypes: object(2)
memory usage: 116.9+ KB


In [ ]:
df_gsm8k.shape

(7472, 2)

In [ ]:
missing_values(df=df_gsm8k, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
question,0,0.0
answer,0,0.0


### Preprocessing Datasets

#### ARC-TR

In [ ]:
# Sözlük formatındaki `choices` sütununu ikiye ayırma
df_arc['choices'] = df_arc['choices'].apply(lambda x: x['text'] if isinstance(x, dict) else x)

In [ ]:
# Seçenekleri ayrı sütunlara ayırmak choice_1,2,3,4
for i in range(4):
    df_arc[f'choice_{i+1}'] = df_arc['choices'].apply(lambda x: x[i] if i < len(x) else None)

In [ ]:
# Doğru cevabın metin olarak eklenmesi
def get_correct_answer(row):
    try:
        index = ord(row["answerKey"]) - ord("A")  # 'A', 'B', 'C', 'D' için 0, 1, 2, 3 üretir
        return row[f'choice_{index + 1}']  # İlgili choice sütununu seçer
    except (KeyError, TypeError):
        return None  # Geçersiz veya eksik veri durumunda None döner

df_arc['correct_answer_text'] = df_arc.apply(get_correct_answer, axis=1)

In [ ]:
df_arc.head()

,id,question,choices,answerKey,choice_1,choice_2,choice_3,choice_4,correct_answer_text
0,Mercury_7175875,"Bir gökbilimci, bir göktaşının çarpmasından so...","[Gezegen yoğunluğu azalacak., Gezegensel yılla...",C,Gezegen yoğunluğu azalacak.,Gezegensel yıllar daha uzun olacak.,Gezegensel günler kısalacak.,Gezegensel yerçekimi daha güçlü hale gelecektir.,Gezegensel günler kısalacak.
1,Mercury_SC_409171,"Bir grup mühendis, farklı bina tasarımlarının ...","[binalar daha hızlı inşa edilecek, binalar dah...",B,binalar daha hızlı inşa edilecek,binalar daha güvenli hale getirilecek,bina tasarımları daha güzel görünecek,inşaat malzemeleri daha ucuz olacak,binalar daha güvenli hale getirilecek
2,Mercury_SC_408547,Fotosentez sürecinin nihai sonucu şeker ve oks...,"[Kimyasal enerji kökler aracılığıyla emilir., ...",C,Kimyasal enerji kökler aracılığıyla emilir.,Işık enerjisi kimyasal enerjiye dönüştürülür.,Yapraktaki klorofil ışık enerjisini yakalar.,Güneş ışığı klorofile dönüştürülür.,Yapraktaki klorofil ışık enerjisini yakalar.
3,Mercury_407327,"Bir fizikçi, bir arabanın bir rampadan atlamak...","[kontrol, hipotez beyanı, bağımlı (yanıt veren...",D,kontrol,hipotez beyanı,bağımlı (yanıt veren) değişken,bağımsız (manipüle edilen) değişken,bağımsız (manipüle edilen) değişken
4,MCAS_2006_9_44,Bir astronot Ay'a 1.0 kg'lık bir cisim ve 5.0 ...,"[Her ikisi de kinetik enerji kaybetti., Her bi...",D,Her ikisi de kinetik enerji kaybetti.,Her biri aynı miktarda potansiyel enerji kazan...,Her ikisi de aynı miktarda potansiyel enerji k...,Her biri maksimum kinetik enerjilerinin yarısı...,Her biri maksimum kinetik enerjilerinin yarısı...


In [ ]:
# `choices` sütununu artık ihtiyaç yoksa kaldırmak
df_arc.drop(columns=['choices'], inplace=True)
df_arc.to_csv(os.path.join(RAW_DATA_PATH, "arc_processed.csv"), index=False, encoding='utf-8')

#### Hellaswag-TR

In [ ]:
# Seçenekleri ayrı sütunlara ayırmak choice_1,2,3,4
for i in range(4):
    df_hellas[f'choice_{i+1}'] = df_hellas['endings'].apply(lambda x: x[i] if i < len(x) else None)

In [ ]:
# Doğru cevabın metin olarak eklenmesi
def get_correct_answer(row):
    try:
        index = ord(row["label"]) - ord("0")  # '0', '1', '2', '3' için 0, 1, 2, 3 üretir (string to int)
        return row[f'choice_{index + 1}']  # İlgili choice sütununu seçer
    except (KeyError, TypeError):
        return None  # Eksik veya geçersiz veri için None döner

df_hellas['correct_answer_text'] = df_hellas.apply(get_correct_answer, axis=1)

In [ ]:
df_hellas.head()

,ind,activity_label,ctx_a,ctx_b,ctx,endings,source_id,split,split_type,label,choice_1,choice_2,choice_3,choice_4,correct_answer_text
0,8,Kurabiye pişirmek,"Beyaz üniformalı bir kadın şef, büyük bir mutf...",tavalar,"Beyaz üniformalı bir kadın şef, büyük bir mutf...","[yumurta sarısı ve kabartma tozu içerir., daha...",activitynet~v_-2dxp-mv2zo,train,indomain,3,yumurta sarısı ve kabartma tozu içerir.,daha sonra esmer şeker serpilir.,tezgahın üzerindeki süzgeç içerisine yerleştir...,hamur işleri ile doldurulur ve fırına yüklenir.,hamur işleri ile doldurulur ve fırına yüklenir.
1,9,Kurabiye pişirmek,"Beyaz üniformalı bir kadın şef, büyük bir mutf...",Bir bıçak,"Beyaz üniformalı bir kadın şef, büyük bir mutf...",[bir tahta üzerinde hareket ettiği ve içindeki...,activitynet~v_-2dxp-mv2zo,train,indomain,3,bir tahta üzerinde hareket ettiği ve içindekil...,"soyulmuş cheesecake, ardından dilimlenmiş muha...",pişmiş tavaların iç kısmına bir şekil kazınır.,Silindir şeklindeki hamurların yuvarlak şekild...,Silindir şeklindeki hamurların yuvarlak şekild...
2,12,Kurabiye pişirmek,Fırına bir tepsi patates konulur ve çıkarılır....,büyük bir tepsi et,Fırına bir tepsi patates konulur ve çıkarılır....,"[fırında patatesin üzerine yerleştirilir., , l...",activitynet~v_-2dxp-mv2zo,train,indomain,3,fırında patatesin üzerine yerleştirilir.,", ls ve turşular fırına konulur.",bir çöplüğe dökülür.,Hazırlandıktan sonra bir yardımcı tarafından f...,Hazırlandıktan sonra bir yardımcı tarafından f...
3,27,Saç kestirmek,Ortadaki adam mavi gömlek giyen kişinin saç st...,mavi gömlekli adam,Ortadaki adam mavi gömlek giyen kişinin saç st...,[mavi gömlek giyen kişinin saçını kesen sünger...,activitynet~v_-JqLjPz-07E,train,indomain,2,mavi gömlek giyen kişinin saçını kesen süngeri...,saç stilini eliyle ve saç spreyiyle yapıyor.,lavabonun yanındaki sandalyeye oturuyor.,göz göze gösteriliyor.,lavabonun yanındaki sandalyeye oturuyor.
4,29,Saç kestirmek,Mavi gömlekli adam lavabonun yanındaki sandaly...,o,Mavi gömlekli adam lavabonun yanındaki sandaly...,"[su musluğunu lavaboya düşürür., sonra lavaboy...",activitynet~v_-JqLjPz-07E,train,indomain,3,su musluğunu lavaboya düşürür.,sonra lavaboya doğru yürüyor ve ıslak saçların...,suyu açar ve sandalyeden kalkar.,şampuanla ovalar ve sonra yıkar.,şampuanla ovalar ve sonra yıkar.


In [ ]:
# `choices` sütununu artık ihtiyaç yoksa kaldırmak
df_hellas = df_hellas.drop(columns=['endings'])
df_hellas.to_csv(os.path.join(RAW_DATA_PATH, "hellaswag_processed.csv"), index=False, encoding='utf-8')

#### GSM8K-TR

In [ ]:
#answer içinde hem nihai sonuç hem de sonuca ulaşmak için gerekli sözel metinler yer aldığı için ikiye böldüm.
df_gsm8k[['answer_method', 'final_answer']] = df_gsm8k['answer'].str.split('####', n=1, expand=True)

# Boşlukları temizlemek
df_gsm8k['answer_method'] = df_gsm8k['answer_method'].str.strip()
df_gsm8k['final_answer'] = df_gsm8k['final_answer'].str.strip()

In [ ]:
df_gsm8k.head()

,question,answer,answer_method,final_answer
0,Weng bebek bakıcılığından saatte 12 dolar kaza...,"Weng dakikada 12/60 = $<<12/60=0,2>>0,2 kazanı...","Weng dakikada 12/60 = $<<12/60=0,2>>0,2 kazanı...",10
1,"Betty, maliyeti 100 dolar olan yeni bir cüzdan...",Başlangıçta Betty'nin elinde sadece 100/2 = $<...,Başlangıçta Betty'nin elinde sadece 100/2 = $<...,5
2,Julie 120 sayfalık bir kitap okuyor. Dün 12 sa...,Maila bugün 12 x 2 = <<12*2=24>>24 sayfa okudu...,Maila bugün 12 x 2 = <<12*2=24>>24 sayfa okudu...,42
3,James haftada iki kez 2 farklı arkadaşına 3 sa...,Her arkadaşına haftada 3*2=<<3*2=6>>6 sayfa ya...,Her arkadaşına haftada 3*2=<<3*2=6>>6 sayfa ya...,624
4,Mark'ın çiçeklerle dolu bir bahçesi var. İçine...,Sarı çiçeklerden 80/100 * 10 = <<80/100*10=8>>...,Sarı çiçeklerden 80/100 * 10 = <<80/100*10=8>>...,35


In [ ]:
df_gsm8k.to_csv(os.path.join(RAW_DATA_PATH, "gsm8k_processed.csv"), index=False, encoding='utf-8')

### Load Datasets: Arc, Hellaswag, GSM8K (main)

In [ ]:
arc_csv_path = os.path.join(RAW_DATA_PATH, "arc_processed.csv")
hellaswag_csv_path = os.path.join(RAW_DATA_PATH, "hellaswag_processed.csv")
gsm8k_csv_path = os.path.join(RAW_DATA_PATH, "gsm8k_processed.csv")

NameError: name 'os' is not defined

In [ ]:
df_arc = pd.read_csv(arc_csv_path)
df_hellaswag = pd.read_csv(hellaswag_csv_path)
df_gsm8k = pd.read_csv(gsm8k_csv_path)

### Split Data

Bağlam için 10 adet örnek,
Test için 10 örneği içermeyen 400 adet örnek

In [ ]:
def prepare_context_and_test_sets(dataset, context_size=10, test_size=400, output_prefix="dataset"):
    # Bağlam için rastgele örnekler seç
    context_samples = dataset.sample(n=context_size, random_state=42)

    # Test seti için bağlam örneklerini çıkart ve rastgele örnekler seç
    remaining_samples = dataset.drop(context_samples.index)
    test_samples = remaining_samples.sample(n=test_size, random_state=42)

    # Bağlam ve test setlerini kaydet
    context_samples.to_csv(os.path.join(RAW_DATA_PATH, f"{output_prefix}_context.csv"), index=False, encoding='utf-8')
    test_samples.to_csv(os.path.join(RAW_DATA_PATH, f"{output_prefix}_test.csv"), index=False, encoding='utf-8')

    print(f"Bağlam ve test setleri '{output_prefix}_context.csv' ve '{output_prefix}_test.csv' olarak kaydedildi.")
    return context_samples, test_samples

In [ ]:
context_arc, test_arc = prepare_context_and_test_sets(df_arc, output_prefix="arc")

Bağlam ve test setleri 'arc_context.csv' ve 'arc_test.csv' olarak kaydedildi.


In [ ]:
context_hellaswag, test_hellaswag = prepare_context_and_test_sets(df_hellaswag, output_prefix="hellaswag")

Bağlam ve test setleri 'hellaswag_context.csv' ve 'hellaswag_test.csv' olarak kaydedildi.


In [ ]:
context_gsm8k, test_gsm8k = prepare_context_and_test_sets(df_gsm8k, output_prefix="gsm8k")

Bağlam ve test setleri 'gsm8k_context.csv' ve 'gsm8k_test.csv' olarak kaydedildi.


### Load splitted data (if require after split data)

In [3]:
def load_splitted_data(output_prefix, output_path="RAW_DATA_PATH"):

    # Path oluşturma
    context_path = os.path.join(output_path, f"{output_prefix}_context.csv")
    test_path = os.path.join(output_path, f"{output_prefix}_test.csv")

    # Dosyaları okuma
    context_samples = pd.read_csv(context_path)
    test_samples = pd.read_csv(test_path)

    print(f"{output_prefix} veri kümesi yüklendi.")
    return context_samples, test_samples

In [4]:
context_arc, test_arc = load_splitted_data("arc", output_path=RAW_DATA_PATH)
context_hellaswag, test_hellaswag = load_splitted_data("hellaswag", output_path=RAW_DATA_PATH)
context_gsm8k, test_gsm8k = load_splitted_data("gsm8k", output_path=RAW_DATA_PATH)

arc veri kümesi yüklendi.
hellaswag veri kümesi yüklendi.
gsm8k veri kümesi yüklendi.


In [ ]:
context_arc = context_arc[['question','correct_answer_text']]
test_arc = test_arc[['question','correct_answer_text']]

context_arc.rename(columns={'question': 'soru'}, inplace=True)
context_arc.rename(columns={'correct_answer_text': 'cevap'}, inplace=True)

test_arc.rename(columns={'question': 'soru'}, inplace=True)
test_arc.rename(columns={'correct_answer_text': 'cevap'}, inplace=True)

<ipython-input-14-60a048dfff54>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_arc.rename(columns={'question': 'soru'}, inplace=True)
<ipython-input-14-60a048dfff54>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_arc.rename(columns={'correct_answer_text': 'cevap'}, inplace=True)
<ipython-input-14-60a048dfff54>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_arc.rename(columns={'question': 'soru'}, inplace=True

In [ ]:
context_arc

,id,question,answerKey,choice_1,choice_2,choice_3,choice_4,correct_answer_text
0,Mercury_177870,Evrendeki tüm yıldızlar bir yaşam döngüsünden ...,C,yıldızda mevcut olan radyasyon,yıldızın tükettiği enerji miktarı,yıldızın başladığı kütle miktarı,yıldızın evrendeki konumu,yıldızın başladığı kütle miktarı
1,Mercury_SC_413439,Yerçekiminin hareket eden nesneler üzerindeki ...,A,aynı yükseklikten düşen kalemler,aynı yükseklikten yukarı doğru süzülen balonlar,düz bir yüzeyde hızlanan bir bisiklet,düz bir yüzey üzerinde yavaşlayan bir top,aynı yükseklikten düşen kalemler
2,MDSA_2011_5_23,Bir öğrenci suyun Dünya yüzeyini nasıl değişti...,D,bir nehir besin ağı diyagramı,bir göldeki yağış miktarının grafiği,bir dağ vadisini kaplayan sisin resmi,yamaçtan aşağı akan bir dere modeli,yamaçtan aşağı akan bir dere modeli
3,Mercury_7205608,"Kimyasal ayrışma, kayalardaki mineraller kimya...",B,hava sıcaklığında düşüş,yağış miktarlarında artış,bir buzulun yavaş hareketi,bitki köklerinin hızlı büyümesi,yağış miktarlarında artış
4,TIMSS_2007_8_pg60,Aşağıdaki hayvanlardan hangisi Dünya'da en uzu...,C,İnsanlar,Kuşlar,Balık,Sürüngenler,Balık
5,Mercury_SC_401585,Gün batımı en geç hangi mevsimde gerçekleşir?,A,yaz,sonbahar,kış,ilkbahar,yaz
6,Mercury_SC_408706,Öğrenciler Dünya'nın Güneş etrafındaki dönüşün...,B,Mart,Haziran,Eylül,Aralık,Haziran
7,Mercury_7038518,Galileo Galilei hangi yasayı tanımlamaktan sor...,D,Maddenin Doğası Yasası,Üç Hareket Yasası,Bağımsız Çeşitlilik Yasası,Güneş Sistemi İçinde Göreli Hareket Yasası,Güneş Sistemi İçinde Göreli Hareket Yasası
8,TAKS_2009_8_32,Aşağıdaki özelliklerden hangisi çevreden en ço...,A,Vücut ağırlığı,Göz rengi,Kan grubu,Renk körlüğü,Vücut ağırlığı
9,Mercury_SC_402029,"Doğada gözlem yaparken, öğrencilerin çevreye s...",D,Yoldan ayrılmayın.,Numune almayın.,Fotoğraf çekin.,Bölgeyi aynen buldukları gibi bıraksınlar.,Bölgeyi aynen buldukları gibi bıraksınlar.


In [ ]:
test_arc

,id,question,answerKey,choice_1,choice_2,choice_3,choice_4,correct_answer_text
0,MEA_2016_8_12,Aşağıdakilerden hangisi ekosistemlerde yavaş b...,B,volkanik patlama,kömür yakmak,orman yangını,petrol döküntüsü,kömür yakmak
1,MCAS_2004_9_2,Lazerler ameliyatlarda şunlar için kullanılır:,A,ince kesikler yapmak.,çalışma alanlarını aydınlatmak.,yaraları dikmek.,aletlerin dezenfekte edilmesi.,ince kesikler yapmak.
2,LEAP_2007_4_10280,Jake soğuk bir günde nefesini dışarı verdiğind...,A,Gaz sıvıya dönüşür.,Katı gaza dönüşür.,Sıvı gaza dönüşür.,Sıvı katıya dönüşür.,Gaz sıvıya dönüşür.
3,Mercury_SC_402984,Hangi madde başka bir fen araştırmasında tekra...,B,lateks eldivenler,bir cam test tüpü,su ve şeker karışımı,temizlemek için kullanılan bir kağıt havlu,bir cam test tüpü
4,Mercury_7270515,Bu sorulardan hangileri bilimsel olarak yanıtl...,B,Hangi müzik notaları insan kulağı tarafından a...,Hangi müzik notaları bir araya geldiğinde en g...,Hangi frekanslar çocuklar tarafından duyulabil...,İnsan ses telleri hangi frekansları üretebilir?,Hangi müzik notaları bir araya geldiğinde en g...
...,...,...,...,...,...,...,...,...
395,Mercury_7037555,Aşağıdakilerden hangisi dışında hepsi güvenli ...,C,düzenli bir masa sağlamak.,deneyler sırasında güvenlik gözlüğü takmak.,talimatları okumadan önce kimyasalları karıştı...,Bir deneyi tamamladıktan sonra elleri iyice yı...,talimatları okumadan önce kimyasalları karıştı...
396,Mercury_402539,"Bir deney sırasında, bir sınıf başlangıçta çev...",B,Balonun içindeki moleküller dışarıya doğru ene...,Balonun içindeki moleküller ısıdan enerji kaza...,Balonun içindeki moleküllerin enerjisi aynı ka...,Balonun içindeki moleküller dışarı kaçıyordu.,Balonun içindeki moleküller ısıdan enerji kaza...
397,NYSEDREGENTS_2015_4_29,Çoğu gıda zincirindeki tüm organizmalar için a...,A,güneş ışığı,su,yeşil bitkiler,ayrıştırıcılar,güneş ışığı
398,Mercury_7158935,Steller deniz aslanı nüfusu Alaska'nın bazı bö...,C,pollock sayısının artırılması,deniz aslanlarinin beslenme düzeninin değiştir...,ringa balığı yaşam alanlarının iyileştirilmesi,deniz aslanlarının yerinin değiştirilmesi,ringa balığı yaşam alanlarının iyileştirilmesi


In [ ]:
context_hellaswag = context_hellaswag[['ctx','correct_answer_text']]
test_hellaswag = test_hellaswag[['ctx','correct_answer_text']]

context_hellaswag.rename(columns={'ctx': 'soru'}, inplace=True)
context_hellaswag.rename(columns={'correct_answer_text': 'cevap'}, inplace=True)

test_hellaswag.rename(columns={'ctx': 'soru'}, inplace=True)
test_hellaswag.rename(columns={'correct_answer_text': 'cevap'}, inplace=True)

<ipython-input-17-27b37984a874>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_hellaswag.rename(columns={'ctx': 'soru'}, inplace=True)
<ipython-input-17-27b37984a874>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_hellaswag.rename(columns={'correct_answer_text': 'cevap'}, inplace=True)
<ipython-input-17-27b37984a874>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_hellaswag.rename(columns={'ctx': 'soru'}, inpl

In [ ]:
context_hellaswag

,ind,activity_label,ctx_a,ctx_b,ctx,source_id,split,split_type,label,choice_1,choice_2,choice_3,choice_4,correct_answer_text
0,27734,Eskrim yapmak,Yeşil korttaki bir odada iki kişi birbirleriyl...,iki kişi,Yeşil korttaki bir odada iki kişi birbirleriyl...,activitynet~v_173d8EtsIpE,train,indomain,2,daha sonra odanın her yerinde bilek güreşi yapın.,yumruk yumruğa ilerleyin ve yumruklarını içeri...,ring boyunca hareket ediyorlar ve birçok kez d...,eylemi saat yönünde gerçekleştiriyorlar.,ring boyunca hareket ediyorlar ve birçok kez d...
1,45451,Evcil Hayvanlar ve Hayvanlar,[header] Köpeğinizin görme yeteneğini kaybedip...,NaN,[header] Köpeğinizin görme yeteneğini kaybedip...,wikihow~74811,train,indomain,2,Köpeğiniz ayrıca içerideki sulu alanlar olan k...,[alt adımlar] Normal gözler birkaç saniye boyu...,"[alt adımlar] Katarakt, bir köpeğin gözbebeği ...",Bir köpek katarakt geliştirdiğinde renk koyula...,"[alt adımlar] Katarakt, bir köpeğin gözbebeği ..."
2,17994,Pencereleri temizleme,Lawrence izleyicilere pencerelerinizi nasıl te...,o,Lawrence izleyicilere pencerelerinizi nasıl te...,activitynet~v_9VGbtQrlcN4,train,indomain,3,tava ve cam bardak gibi kırık cam panellerin n...,duvar kağıdının özelliklerini ve bunlardan bir...,büyük bir bütçe kremi yenilemesi gösteriyor ve...,sürücü camını indirir ve sabun ve su solüsyonu...,sürücü camını indirir ve sabun ve su solüsyonu...
3,48089,Ev ve bahçe,[başlık] Kuru silme tahtasındaki eski izler na...,NaN,[başlık] Kuru silme tahtasındaki eski izler na...,wikihow~79147,train,indomain,1,Ahşap paneli yağ bazlı bir lekeyle kapladıysan...,En iyi sonuçları elde etmek için %99 veya %90 ...,[alt adımlar] Diğer aşındırıcı maddeler de lek...,"Hırdavatçıda tuvalet ispirtosu bulabilirseniz,...",En iyi sonuçları elde etmek için %99 veya %90 ...
4,5868,Spor ve Fitness,[başlık] Yılan ısırığı nasıl tedavi edilir [ba...,NaN,[başlık] Yılan ısırığı nasıl tedavi edilir [ba...,wikihow~9721,train,indomain,1,"Dikkat: Malzemeleri ısırabilir, ahşabı ve diğe...",İlk müdahale ekipleri bölgede bulunan yılan tü...,Bir yılan tarafından ısırılırsanız veya ısırıl...,Personele arayabileceğiniz bir hastane veya ac...,İlk müdahale ekipleri bölgede bulunan yılan tü...
5,44715,Tatiller ve Gelenekler,[header] Çin ay festivalinin tadını nasıl çıka...,NaN,[header] Çin ay festivalinin tadını nasıl çıka...,wikihow~73624,train,indomain,3,"Ailenizle, arkadaşlarınızla ve ailenizle festi...","Ay festivali, ayın haftasının gece ve tatil ge...","Festival, balık, meyve, balık yağı, baharat, b...","Festival, Çin takvimine göre sekizinci ayın on...","Festival, Çin takvimine göre sekizinci ayın on..."
6,11676,Bilgisayarlar ve Elektronik,[header] Samsung galaxy s4'ten silinen kısa me...,NaN,[header] Samsung galaxy s4'ten silinen kısa me...,wikihow~19253,train,indomain,1,[alt adımlar] Verilerin eksik olduğunu fark ed...,"Bu adım çok önemlidir, bu yüzden göz ardı etme...",[alt adımlar] Bunun yerine telefonunuzun ayarl...,"Samsung galaxy s4, yedekleme moduna sahip ciha...","Bu adım çok önemlidir, bu yüzden göz ardı etme..."
7,41388,Arabalar ve Diğer Araçlar,[başlık] Kaliforniya'da bir araba nasıl kayded...,NaN,[başlık] Kaliforniya'da bir araba nasıl kayded...,wikihow~68114,train,indomain,1,[alt adımlar] Arabanızı kaydettirmek için hem ...,[alt adımlar] Arabanızı bir bayiden satın aldı...,[alt adımlar] Arabanızı Kaliforniya'ya götürme...,"[alt adımlar] Bu formlar, aracınızı nasıl kayd...",[alt adımlar] Arabanızı bir bayiden satın aldı...
8,20523,Eğitim ve İletişim,[header] Beğendiğiniz bir çocuğa nasıl not yaz...,NaN,[header] Beğendiğiniz bir çocuğa nasıl not yaz...,wikihow~33861,train,indomain,0,Ona mesaj gönderebilmen için sana numarasını v...,"Muhtemelen bir okul danışmanından, terapistten...","Şu anda ana fikri açıklamanıza gerek yok, sade...","""Seni seviyorum"" gibi daha derin bir şeyler ya...",Ona mesaj gönderebilmen için sana numarasını v...
9,5518,Disk atma,Bir adamın daireler çizerek döndüğü ve uzaklar...,daha fazla klip,Bir adamın daire

In [ ]:
test_hellaswag

,soru,cevap
0,[header] Gün içinde uykuya nasıl dalılır? [tit...,[alt adımlar] Gün içinde kendinizi tembel hiss...
1,[başlık] Bilgisayar korsanlarından ve korsanla...,Ağınız şifreniz için noktalama işaretlerine iz...
2,Bir adam kol kola yürürken bir kadını yanağınd...,adama evindeki bazı eşyaları verir.
3,Bir adam denge aletinin üzerinde sallanıyor. A...,denge çubuğuna geri döner.
4,[header] Kendinizi nasıl yok edersiniz [title]...,[title] Vücudunuzda kızarıklık olup olmadığına...
...,...,...
395,"Kadın yere yatıyor, sonra bacaklarından baş aş...",kollar tarafından götürülür.
396,[header] Arabanızın termostatının kapalı kalıp...,[adım] Eğer yapabiliyorsanız soğutma sıvısı re...
397,[header] Bir kadınla nasıl yakınlık kurulur? [...,Bu tekniği tanımlamanın en iyi yolu kendinizi ...
398,"Adam biletleri bağırmaya başlıyor. Kadınlar, a...",onlara yardım eder ve onlara yol gösterir.


In [ ]:
context_gsm8k = context_gsm8k[['question','answer','final_answer']]
test_gsm8k = test_gsm8k[['question','answer','final_answer']]

context_gsm8k.rename(columns={'question': 'soru'}, inplace=True)
context_gsm8k.rename(columns={'answer': 'uzun_cevap'}, inplace=True)
context_gsm8k.rename(columns={'final_answer': 'cevap'}, inplace=True)

test_gsm8k.rename(columns={'question': 'soru'}, inplace=True)
test_gsm8k.rename(columns={'answer': 'uzun_cevap'}, inplace=True)
test_gsm8k.rename(columns={'final_answer': 'cevap'}, inplace=True)

<ipython-input-18-79ca340ea58c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_gsm8k.rename(columns={'question': 'soru'}, inplace=True)
<ipython-input-18-79ca340ea58c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_gsm8k.rename(columns={'answer': 'uzun_cevap'}, inplace=True)
<ipython-input-18-79ca340ea58c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_gsm8k.rename(columns={'final_answer': 'cevap'}, inplac

In [ ]:
context_gsm8k

,soru,uzun_cevap,cevap
0,Mistik Akvaryum'da köpekbalıkları her gün dört...,Yunuslar her gün 4/2 = <<4/2=2>>2 kova balıkla...,3
1,Paten derbisinde 4 takım yarışıyor. Her takım ...,Toplamda 4 takım var * takım başına 10 üye = <...,240
2,"2 saat koşan Jonah, saatte 30 kalori yakıyor. ...",Yunus 2 saat boyunca saatte 30 kalori yakarak ...,90
3,Bay McNeely her gün sınıfta bulunan toplam öğr...,Üçüncü gün 1/7*280 = <<1/7*280=40>>40 öğrenci ...,240
4,Joseph'in birçok farklı renkte çorapları vardı...,"6 kırmızı çorapla başlayarak, mavi çorap sayıs...",28
5,İstiridyeyle doymak için Crabby'nin Squido'nun...,"Eğer Squido 200 istiridye yerse, Crabby Squido...",600
6,"Bay Banks ve Bayan Elizabeth, farklı gelir akı...",Bay Banks'in 8 yatırım akışı olduğundan ve her...,500
7,Dorothy cam çerçeveleri Jemma'nın sattığı fiya...,Jemma tanesi 5 dolardan 400 gözlük çerçevesi s...,2500
8,"Amy, mum yapmak için fitilleri kesmek üzere 15...",15 fitlik ip 15*12=<<15*12=180>>180 inç uzunlu...,20
9,"Bir sürahiye 5 litre su, bir kovaya da 4 sürah...",4 sürahi 4*5 = <<4*5=20>>20 litre su içerir.\n...,40


In [ ]:
test_gsm8k

NameError: name 'test_gsm8k' is not defined

### Bağlam oluşturma

Arc ve Hellaswag

In [ ]:
def prepare_arc_context(context_samples, test_question, context_size=10):
    """
    ARC-TR veri kümesi için 10-shot bağlam hazırlar.
    """
    context = ""
    for i in range(context_size):  # İlk 10 örneği bağlam olarak kullan
        context += f"Question {i+1}: {context_samples.iloc[i]['soru']}\n"
        context += f"Answer {i+1}: {context_samples.iloc[i]['cevap']}\n\n"

    # Test sorusunu bağlama ekle
    context += f"Test Question: {test_question}\n"
    return context

In [29]:
def prepare_arc_context(context_samples, test_question, test_choices, context_size=10):
    """
    ARC-TR veri kümesi için 10-shot bağlam hazırlar ve şıkları ve doğru cevabı ekler.
    """
    context = ""
    for i in range(context_size):  # İlk 10 örneği bağlam olarak kullan
        question = context_samples.iloc[i]['question']
        choices = [
            context_samples.iloc[i]['choice_1'],
            context_samples.iloc[i]['choice_2'],
            context_samples.iloc[i]['choice_3'],
            context_samples.iloc[i]['choice_4']
        ]
        correct_choice = context_samples.iloc[i]['answerKey']  # 0, 1, 2, 3 olarak normalize edildi
        correct_answer_text = context_samples.iloc[i]['correct_answer_text']

        # Bağlama soruyu, şıkları ve doğru cevabı ekle
        context += f"Question {i+1}: {question}\n"
        context += "Choices:\n"
        for j, choice in enumerate(choices):
            if choice:  # Eğer boş değilse ekle
                context += f"{chr(65 + j)}) {choice}\n"
            else:
                context += f"{chr(65 + j)}) [NO CHOICE]\n"  # Boş olanlara bilgi ekle
        context += f"Correct Answer: {chr(65 + correct_choice)}) {correct_answer_text}\n\n"


    # Test sorusunu bağlama ekle
    context += f"Test Question: {test_question}\n"
    context += "Choices:\n"
    for j, choice in enumerate(test_choices):
        if choice:
            context += f"{chr(65 + j)}) {choice}\n"
        else:
            context += f"{chr(65 + j)}) [NO CHOICE]\n"
    return context

In [ ]:
def prepare_hellaswag_context(context_samples, test_ctx, test_choices, context_size=10):
    """
    Hellaswag veri kümesi için 10-shot bağlam hazırlar ve test bağlamı ile şıkları ekler.
    """
    context = ""
    for i in range(context_size):  # İlk 10 bağlamı kullan
        ctx = context_samples.iloc[i]['baglam']
        choices = [
            context_samples.iloc[i]['choice_1'],
            context_samples.iloc[i]['choice_2'],
            context_samples.iloc[i]['choice_3'],
            context_samples.iloc[i]['choice_4']
        ]
        correct_label = context_samples.iloc[i]['label']  # Örneğin 0, 1, 2, 3
        correct_answer_text = context_samples.iloc[i]['correct_answer_text']

        # Bağlama bağlamı, şıkları ve doğru cevabı ekle
        context += f"Context {i+1}: {ctx}\n"
        context += "Choices:\n"
        for j, choice in enumerate(choices):
            context += f"{chr(65 + j)}) {choice}\n"
        context += f"Correct Ending: {chr(65 + correct_label)}) {correct_answer_text}\n\n"

    # Test bağlamını ekle
    context += f"Test Context: {test_ctx}\n"
    context += "Choices:\n"
    for j, choice in enumerate(test_choices):
        context += f"{chr(65 + j)}) {choice}\n"
    return context


Gsm8k

In [ ]:
def prepare_gsm8k_context(dataset, test_question, context_size=10):
    context = ""
    for i in range(context_size):  # Bağlam için ilk 10 örneği kullan
        context += f"Question {i+1}: {dataset.iloc[i]['soru']}\n"
        context += f"Answer {i+1}: {dataset.iloc[i]['uzun_cevap']} #### {dataset.iloc[i]['cevap']}\n\n"
    context += f"Test Question: {test_question}\n"
    return context

### Load Model: cosmos-DPO

In [12]:
model_id = "ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1"
model_cache_dir = os.path.join(CACHE_PATH, "cosmos_dpo")

# Tokenizer ve modelin yüklenmesi (cache kontrolü)
if not os.path.exists(model_cache_dir):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,  # Daha az bellek tüketimi için bf16
    )
    # Model ve tokenizer'ı cache dizinine kaydet
    tokenizer.save_pretrained(model_cache_dir)
    model.save_pretrained(model_cache_dir)
else:
    # Cache'den yükleme
    tokenizer = AutoTokenizer.from_pretrained(model_cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        model_cache_dir,
        torch_dtype=torch.bfloat16,  # Daha az bellek tüketimi için bf16
    )

# Cihazı ayarla (CUDA veya CPU)
model = model.to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Prediction

In [27]:
def get_prediction(context, model, tokenizer, device):
    """
    Bağlamı alır ve Cosmos-DPO modelinden tahmin edilen cevabı döndürür.
    """
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Giriş metnini işleme
    inputs = tokenizer(
        context,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024  # Giriş uzunluğunu sınırla
    ).to(device)

    # Tahmin üretme
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=256,  # Üretilecek metin uzunluğunu sınırla
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,  # Sonlandırma tokeni
        temperature=0.6,  # Rastgeleliği kontrol et
        top_p=0.9         # Top-p çekimi (nükleus sampling)
    )

    # Tahmini çözümleme
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return predicted_answer


In [25]:
from tqdm import tqdm

def run_predictions_arc(context_samples, test_samples, model, tokenizer, device):
    """
    ARC-TR veri kümesi üzerinde tahminler alır.
    """
    # Boş değerleri doldurma ve veri tiplerini normalize etme
    test_samples[['choice_1', 'choice_2', 'choice_3', 'choice_4']] = test_samples[['choice_1', 'choice_2', 'choice_3', 'choice_4']].fillna("").astype(str)
    context_samples[['choice_1', 'choice_2', 'choice_3', 'choice_4']] = context_samples[['choice_1', 'choice_2', 'choice_3', 'choice_4']].fillna("").astype(str)

    predictions = []
    for idx, row in tqdm(test_samples.iterrows(), total=len(test_samples), desc="Tahmin Alınıyor"):
        # Test sorusu için bağlamı hazırla
        context = prepare_arc_context(
            context_samples,
            row['question'],
            [row['choice_1'], row['choice_2'], row['choice_3'], row['choice_4']]
        )

        # Tahmini al
        predicted_answer = get_prediction(context, model, tokenizer, device)

        # Tahmini metinden şık olarak eşleştirme
        predicted_choice = None
        for i, choice in enumerate([row['choice_1'], row['choice_2'], row['choice_3'], row['choice_4']]):
            if isinstance(choice, str) and predicted_answer.strip().lower() == choice.strip().lower():
                predicted_choice = i  # 0, 1, 2, 3 olarak kaydet

        # Tahmini kaydet
        predictions.append({
            "question": row['question'],
            "correct_answer": row['correct_answer_text'],  # Doğru cevabın metinsel içeriği
            "predicted_answer": predicted_answer,          # Tahmin edilen metin
            "correct_choice": row['answerKey'],            # Doğru şık (0, 1, 2, 3)
            "predicted_choice": predicted_choice,          # Tahmin edilen şık (0, 1, 2, 3)
            "context": context
        })

    # Sonuçları döndür
    return pd.DataFrame(predictions)



In [ ]:
def calculate_accuracy(results):
    """
    Tahmin edilen cevaplarla doğru cevapları karşılaştırarak doğruluk hesaplar.
    """
    correct_count = (results["correct_answer"] == results["predicted_answer"]).sum()
    accuracy = correct_count / len(results)
    print(f"Top-1 Accuracy: {accuracy:.2%}")

In [26]:
def calculate_accuracy_by_choice(results):
    """
    Tahmin edilen şık ile doğru şık üzerinden doğruluk hesaplar.
    """
    correct_count = (results["predicted_choice"] == results["correct_choice"]).sum()
    accuracy = correct_count / len(results)
    print(f"Top-1 Accuracy (Choice-based): {accuracy:.2%}")


In [ ]:
print("Pad Token:", tokenizer.pad_token)
print("Pad Token ID:", tokenizer.pad_token_id)

Pad Token: None
Pad Token ID: None


In [30]:
# Tahminleri al
results_arc = run_predictions_arc(context_arc, test_arc, model, tokenizer, device)

# Sonuçları kaydet
results_arc.to_csv(os.path.join(DATA_PATH, "arc_results.csv"), index=False, encoding='utf-8')

# Doğruluğu hesapla
calculate_accuracy_by_choice(results_arc)

Tahmin Alınıyor: 100%|██████████| 400/400 [1:03:38<00:00,  9.55s/it]

Top-1 Accuracy (Choice-based): 0.00%


In [9]:
print(test_arc[['choice_1', 'choice_2', 'choice_3', 'choice_4']].isnull().sum())

choice_1    0
choice_2    0
choice_3    0
choice_4    3
dtype: int64


In [ ]:
# Boş değer içeren satırları bulma
missing_choice_4 = test_arc[test_arc['choice_4'].isnull()]

# Boş değer içeren satırların index bilgileri
print("Boş Değer İçeren Satırların İndeksleri:")
print(missing_choice_4.index)

# Boş değer içeren satırların id bilgileri (eğer 'id' sütunu varsa)
if 'id' in test_arc.columns:
    print("Boş Değer İçeren Satırların ID'leri:")
    print(missing_choice_4['id'].tolist())

Boş Değer İçeren Satırların İndeksleri:
Index([97, 194, 313], dtype='int64')
Boş Değer İçeren Satırların ID'leri:
['NYSEDREGENTS_2015_4_7', 'NYSEDREGENTS_2012_8_42', 'NYSEDREGENTS_2010_4_4']


In [10]:
# Boş değerleri doldurma
test_arc['choice_4'] = test_arc['choice_4'].fillna("")
context_arc['choice_4'] = context_arc['choice_4'].fillna("")

In [11]:
print(test_arc[['choice_1', 'choice_2', 'choice_3', 'choice_4']].isnull().sum())

choice_1    0
choice_2    0
choice_3    0
choice_4    0
dtype: int64


In [23]:
# check all answerKey in test_arc and if even one key is a digit then print that indix of key. And print the key also.
for i in range(len(test_arc)):
    if test_arc['answerKey'][i].isdigit():
        print(i, test_arc['answerKey'][i])

AttributeError: 'numpy.int64' object has no attribute 'isdigit'

In [21]:
def normalize_answer_key(answer_key):
    """
    ARC veri setindeki answerKey sütununu normalize eder.
    """
    try:
        if answer_key.isdigit():  # Sayı ise
            return int(answer_key) - 1  # 1, 2, 3, 4 -> 0, 1, 2, 3
        else:  # Harf ise
            return ord(answer_key) - ord('A')  # A, B, C, D -> 0, 1, 2, 3
    except:
        return None  # Geçersiz değer varsa


In [22]:
# answerKey sütununu normalize et
test_arc['answerKey'] = test_arc['answerKey'].apply(normalize_answer_key)
context_arc['answerKey'] = context_arc['answerKey'].apply(normalize_answer_key)